In [1]:
from PIL import Image
import numpy as np
import os
from tqdm import tqdm
import sqlite3

In [2]:
def ricomponi_mappa(cartella):
    # Definisci le dimensioni di ciascuna immagine e la griglia totale
    larghezza_img = 256
    altezza_img = 256
    righe = 32
    colonne = 128
    
    # Dimensioni dell'immagine finale
    larghezza_totale = larghezza_img * colonne
    altezza_totale = altezza_img * righe
    
    # Crea una nuova immagine vuota con le dimensioni totali
    immagine_finale = Image.new('RGB', (larghezza_totale, altezza_totale))
    
    # Loop attraverso tutte le posizioni per inserire le immagini
    for riga in range(16, 48):
        for colonna in range(0, 128):
            # Costruisci il nome del file
            nome_file = f"image_{riga}_{colonna}.png"
            percorso_file = os.path.join(cartella, nome_file)
            
            # Carica l'immagine
            if os.path.exists(percorso_file):
                img = Image.open(percorso_file)
                # Calcola la posizione in cui incollare l'immagine
                posizione_x = (colonna) * larghezza_img
                posizione_y = (riga - 16) * altezza_img
                
                # Incolla l'immagine nel punto corretto
                immagine_finale.paste(img, (posizione_x, posizione_y))
            else:
                print(f"File {percorso_file} non trovato.")
    
    return immagine_finale

In [3]:
def immagine_a_matrice_parziale(immagine, start_row, end_row):
    # Converti l'immagine in formato RGB (se non lo è già)
    immagine = immagine.convert('RGB')
    
    # Ottieni i dati dell'immagine come array NumPy
    array_dati = np.array(immagine)
    
    # Ottieni la porzione specificata dell'immagine
    porzione_dati = array_dati[start_row:end_row]
    
    return porzione_dati

def salva_matrice_in_db_parziale(matrice, start_row, nome_db):
    conn = sqlite3.connect(nome_db)
    cur = conn.cursor()
    
    # Crea la tabella per i pixel se non esiste già
    cur.execute('''
        CREATE TABLE IF NOT EXISTS pixels (
            x INTEGER,
            y INTEGER,
            value INTEGER
        )
    ''')
    
    # Inserisce i dati della matrice nel database
    altezza, larghezza, _ = matrice.shape
    
    for i in tqdm(range(altezza), desc=f"Salvataggio della matrice nel DB dalla riga {start_row}"):
        for j in range(larghezza):
            if len(matrice.shape) == 3:  # Se l'immagine è a colori
                r, g, b = matrice[i, j]
                value = int(0.2989 * r + 0.587 * g + 0.114 * b)  # Calcolo del valore di intensità grigia
            else:  # Se l'immagine è in scala di grigi
                value = int(matrice[i, j])
            cur.execute('INSERT INTO pixels (x, y, value) VALUES (?, ?, ?)', (start_row + i, j, value))
    
    # Commit e chiusura della connessione
    conn.commit()
    conn.close()

def processa_immagine_in_tranche(cartella, nome_db, num_tranche=4):
    immagine_ricomposta = ricomponi_mappa(cartella)
    altezza_totale = immagine_ricomposta.size[1]
    
    tranche_size = altezza_totale // num_tranche
    
    for tranche in range(num_tranche):
        start_row = tranche * tranche_size
        end_row = (tranche + 1) * tranche_size if tranche < num_tranche - 1 else altezza_totale
        
        matrice_parziale = immagine_a_matrice_parziale(immagine_ricomposta, start_row, end_row)
        salva_matrice_in_db_parziale(matrice_parziale, start_row, nome_db)

In [4]:
cartella_programma = "tante_immagini"
nome_db = 'matrice_pixel.db'
processa_immagine_in_tranche(cartella_programma, nome_db)

Salvataggio della matrice nel DB dalla riga 6144: 100%|██████████| 2048/2048 [04:44<00:00,  7.21it/s]


In [5]:
#sotto_matrice = estrai_sotto_matrice(matrice, n, m, l)
#print(sotto_matrice)